# Proyecto Puzzle-8

**Solución al proyecto *Puzzle-8*** del Prof. Carlos Ogando para la asignatura Introducción a la IA en ITLA


**Formato de entrada:**  
*{algoritmo}* `(bfs / dfs / ast)` + *{estado inicial}*  

**Ejemplo:**  
bfs 0,1,2,3,4,8,7,6,5

In [1]:
"""
Código esqueleto para el proyecto Puzzle-8 del Prof. Carlos Ogando
para la asignatura Introducción a la IA en ITLA.
Python 3
"""

# Librerías

import queue as Q

import time

import math

import psutil

import functools


In [2]:
# Clase que representa el Puzzle-n general

class PuzzleState(object):

    """docstring para PuzzleState"""

    def __init__(self, config, n, parent=None, action="Initial", cost=0):

        if n*n != len(config) or n < 2:

            raise Exception("the length of config is not correct!")

        self.n = n

        self.cost = cost

        self.parent = parent

        self.action = action

        self.dimension = n

        self.config = config

        self.children = []

        for i, item in enumerate(self.config):

            if item == 0:

                self.blank_row = i // self.n

                self.blank_col = i % self.n

                break

    def display(self):

        for i in range(self.n):

            line = []

            offset = i * self.n

            for j in range(self.n):

                line.append(self.config[offset + j])

            print(line)

    def move_left(self):

        if self.blank_col == 0:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index - 1

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Left", cost=self.cost + 1)

    def move_right(self):

        if self.blank_col == self.n - 1:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index + 1

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Right", cost=self.cost + 1)

    def move_up(self):

        if self.blank_row == 0:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index - self.n

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Up", cost=self.cost + 1)

    def move_down(self):

        if self.blank_row == self.n - 1:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index + self.n

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Down", cost=self.cost + 1)

    def expand(self):

        """Expandir el nodo"""

        # Añadir nodos hijos en orden UDLR (Up-Down-Left-Right)

        if len(self.children) == 0:

            up_child = self.move_up()

            if up_child is not None:

                self.children.append(up_child)

            down_child = self.move_down()

            if down_child is not None:

                self.children.append(down_child)

            left_child = self.move_left()

            if left_child is not None:

                self.children.append(left_child)

            right_child = self.move_right()

            if right_child is not None:

                self.children.append(right_child)

        return self.children

In [3]:
# Clases que representan a una frontera consultable
# (Esto es porque la clase queue en Python no es consultable)

# QUEUE

class QueueFrontier():

    def __init__(self):

      self.data = []

      self.configs = set()

    def push(self, value):

      self.data.append(value)

      self.configs.add(value.config)

    def pop(self):

      value = self.top()

      self.data = self.data[1:]

      self.configs.remove(value.config)

      return value

    def top(self):

      return self.data[0]

    def empty(self):

      return len(self.data) == 0

    def __len__(self):

      return len(self.data)

    def __contains__(self, value):

      return value.config in self.configs

# STACK

class StackFrontier(QueueFrontier):

    def pop(self):

      value = self.top()

      self.data = self.data[:-1]

      return value

    def top(self):

      return self.data[-1]

# PRIORITY QUEUE

class PriorityQueueFrontier(QueueFrontier):

    def push(self, value):

      self.data.append(value)

      self.configs.add(value[1].config)

      self.updateQueue()

    def pop(self):

      value = self.top()

      self.data = self.data[1:]

      self.configs.remove(value[1].config)

      return value

    def updateQueue(self):

      self.data.sort(key = lambda x : x[0])

    def decreaseKey(self, value): # USELESS FOR THIS GAME

      #pos = [state.config for priority, state in self.data].index(value[1].config)

      #if value[0] < self.data[pos][0]:

        #self.data[pos] = value

        #self.updateQueue()

      pass



In [4]:
# Variables globales

PATH_TO_GOAL = None

COST_OF_PATH = 0

NODES_EXPANDED = 0

SEARCH_DEPTH = 0

MAX_SEARCH_DEPTH = 0

RUNNING_TIME = 0

RAM_USAGE = 0


In [ ]:
# Decorador para obtener tiempo y uso de RAM

# Obtenga el valor del tiempo y uso de RAM llamando a las variables globales RUNNING_TIME Y RAM_USAGE
# después de ejecutar cualquiera de los algoritmos

def runningTime(func):

  @functools.wraps(func)
  def wrapper(*args, **kwargs):

    start_time = time.time()

    value = func(*args, **kwargs)

    end_time = time.time()

    #print(end_time - start_time, "segundos")

    global RUNNING_TIME

    RUNNING_TIME = end_time - start_time

    return value

  return wrapper

def ramUsage(func):

  @functools.wraps(func)
  def wrapper(*args, **kwargs):

    #print(psutil.virtual_memory().percent, "% de memoria virtual usada (", psutil.Process().memory_info().rss/10**6, " MB)")

    value = func(*args, **kwargs)

    #print(psutil.virtual_memory().percent, "% de memoria virtual usada (", psutil.Process().memory_info().rss/10**6, " MB)")

    global RAM_USAGE

    RAM_USAGE = psutil.Process().memory_info().rss/10**6

    return value

  return wrapper

In [ ]:
# HELPER FUNCTION

def backtrack(actual_state):

  """ Indica el camino desde el estado inicial hasta el actual_state. """

  path = [actual_state]

  while actual_state.parent != None:

      path.append(actual_state.parent)

      actual_state = actual_state.parent

  path.reverse()

  actions = [state.action for state in path]

  return actions[1:]

In [7]:
def writeOutput():

    global PATH_TO_GOAL, COST_OF_PATH, NODES_EXPANDED, SEARCH_DEPTH, MAX_SEARCH_DEPTH, RUNNING_TIME, RAM_USAGE

    # Escribir en output.txt los valores necesarios

    file = open("output.txt", "w+")

    file.write("path_to_goal: " + str(PATH_TO_GOAL) + "\n")

    file.write("cost_of_path: " + str(COST_OF_PATH) + "\n")

    file.write("nodes_expanded: " + str(NODES_EXPANDED) + "\n")

    file.write("search_depth: " + str(SEARCH_DEPTH) + "\n")

    file.write("max_search_depth: " + str(MAX_SEARCH_DEPTH) + "\n")

    file.write("running_time: " + str(RUNNING_TIME) + "\n")

    file.write("max_ram_usage: " + str(RAM_USAGE) + "\n")

    # Resetear los valores de las variables globales

    COST_OF_PATH = NODES_EXPANDED = SEARCH_DEPTH = MAX_SEARCH_DEPTH = RUNNING_TIME = RAM_USAGE = 0

    PATH_TO_GOAL = None

    file.close()


**Funciones y algoritmos a implementar**

---



In [8]:
@runningTime
@ramUsage
def bfs_search(initial_state):

    """BFS search"""

    global PATH_TO_GOAL, COST_OF_PATH, NODES_EXPANDED, SEARCH_DEPTH, MAX_SEARCH_DEPTH

    frontier = QueueFrontier()

    frontier.push(initial_state)

    explored = set()

    while not frontier.empty():

      state = frontier.pop()

      explored.add(state.config)

      if test_goal(state):

        COST_OF_PATH = SEARCH_DEPTH = state.cost

        PATH_TO_GOAL = backtrack(state)

        return

      for neighbor in state.expand():

        if (neighbor not in frontier) and (neighbor.config not in explored):

          frontier.push(neighbor)

          MAX_SEARCH_DEPTH = max(MAX_SEARCH_DEPTH, neighbor.cost)

      NODES_EXPANDED += 1

    raise Exception("No se encontró solución")


In [9]:
@runningTime
@ramUsage
def dfs_search(initial_state):

    """DFS search"""

    global PATH_TO_GOAL, COST_OF_PATH, NODES_EXPANDED, SEARCH_DEPTH, MAX_SEARCH_DEPTH

    frontier = StackFrontier()

    frontier.push(initial_state)

    explored = set()

    while not frontier.empty():

      state = frontier.pop()

      explored.add(state.config)

      if test_goal(state):

        COST_OF_PATH = SEARCH_DEPTH = state.cost

        PATH_TO_GOAL = backtrack(state)

        return

      for neighbor in reversed(state.expand()):

        if (neighbor not in frontier) and (neighbor.config not in explored):

          frontier.push(neighbor)

          MAX_SEARCH_DEPTH = max(MAX_SEARCH_DEPTH, neighbor.cost)

      NODES_EXPANDED += 1

    raise Exception("No se encontró solución")

In [10]:
@runningTime
@ramUsage
def A_star_search(initial_state):

    """A * search"""

    global PATH_TO_GOAL, COST_OF_PATH, NODES_EXPANDED, SEARCH_DEPTH, MAX_SEARCH_DEPTH

    frontier = PriorityQueueFrontier()

    frontier.push((calculate_total_cost(initial_state), initial_state))

    explored = set()

    while not frontier.empty():

      priority, state = frontier.pop()

      explored.add(state.config)

      if test_goal(state):

        COST_OF_PATH = SEARCH_DEPTH = state.cost

        PATH_TO_GOAL = backtrack(state)

        return

      for neighbor in state.expand():

        if (neighbor not in frontier) and (neighbor.config not in explored):

          heuristic_cost = calculate_total_cost(neighbor)

          frontier.push((heuristic_cost + neighbor.cost, neighbor))

          MAX_SEARCH_DEPTH = max(MAX_SEARCH_DEPTH, neighbor.cost)

        elif neighbor in frontier:

          heuristic_cost = calculate_total_cost(neighbor)

          frontier.decreaseKey((heuristic_cost + neighbor.cost, neighbor))

      NODES_EXPANDED += 1

    raise Exception("No se encontró solución")

In [11]:
def calculate_total_cost(state):

    """calculate the total estimated cost of a state"""

    total = 0

    for idx in range(9):

        val = state.config[idx]

        if(val != 0):

            total += calculate_manhattan_dist(idx, val, state.n)

    return total

In [12]:
def calculate_manhattan_dist(idx, value, n):

    """calculate the manhattan distance of a tile"""

    curr_row = idx // n

    curr_col = idx % n

    target_row = value // n

    target_col = value % n

    return abs(target_row - curr_row) + abs(target_col - curr_col)


In [13]:
def test_goal(puzzle_state):

    """test the state is the goal state or not"""

    return calculate_total_cost(puzzle_state) == 0

In [14]:
# Función Main que leerá las entradas y llamará el algoritmo correspondiente

def main():

    query = input().split(" ")

    sm = query[0].lower()

    begin_state = query[1].split(",")

    begin_state = tuple(map(int, begin_state))

    size = int(math.sqrt(len(begin_state)))

    hard_state = PuzzleState(begin_state, size)

    if sm == "bfs":

        bfs_search(hard_state)

    elif sm == "dfs":

        dfs_search(hard_state)

    elif sm == "ast":

        A_star_search(hard_state)

    else:

        print("Introduzca comandos de argumentos válidos !")

        return

    writeOutput()

if __name__ == '__main__':

    main()

In [15]:
# TESTING PRIORITY QUEUE

class StringState():

  def __init__(self, config):

    self.config = config

  def __repr__(self):

    return self.config

pq = PriorityQueueFrontier()


pq.push((9, StringState("Carlos")))
pq.push((3, StringState("Carmen")))
pq.push((1, StringState("Ruth")))
pq.push((7, StringState("Karla")))
pq.push((6.5, StringState("Luis")))
pq.push((5, StringState("Laura")))

print(pq.data)
pq.decreaseKey((2, StringState("Carlos")))
pq.decreaseKey((1, StringState("Laura")))
print(pq.data)
pq.decreaseKey((5, StringState("Laura")))
pq.decreaseKey((9, StringState("Ruth")))
print(pq.data)
pq.decreaseKey((1, StringState("Carmen")))
pq.decreaseKey((6, StringState("Karla")))
print(pq.data)

[(1, Ruth), (3, Carmen), (5, Laura), (6.5, Luis), (7, Karla), (9, Carlos)]
[(1, Ruth), (3, Carmen), (5, Laura), (6.5, Luis), (7, Karla), (9, Carlos)]
[(1, Ruth), (3, Carmen), (5, Laura), (6.5, Luis), (7, Karla), (9, Carlos)]
[(1, Ruth), (3, Carmen), (5, Laura), (6.5, Luis), (7, Karla), (9, Carlos)]
